In [1]:
import cv2
import numpy as np

In [1]:
my_colour = [[255,255,255],[0,0,0]]
# color_limit = [[0,146,85,10,255,255]]
color_limit = [[0,201,79,6,255,125]]

In [3]:
def findcolor(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_limit = np.array(color_limit[0][0:3])
    upper_limit = np.array(color_limit[0][3:6])
    mask = cv2.inRange(img_hsv, lower_limit, upper_limit)
    cv2.imshow('mask', mask) 
    contour(mask)
    

In [7]:
def contour(img_co):
    contur, hierarcy = cv2.findContours(img_co, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    new_points = []
    for cr in contur :
        area = cv2.contourArea(cr)
        if area > 300:
            cv2.drawContours(img, contur, -1, (0, 0, 255), 3)
            # approx = cv2.approxPolyDP(contur, 0.01*cv2.arcLength(contur, True), True)
            # x,y,w,h = cv2.boundingRect(approx)
            # cv2.circle(img, (x+w/2,y+h/2), 15 , my_colour[0], -1)
            # if x!=0 and y!=0:
            #     new_points.append([x,y])
            

In [8]:
def video():
    framwidth = 640
    framheight = 480
    cap = cv2.VideoCapture(0)
    cap.set(3, framwidth)
    cap.set(4, framheight)
    cap.set(10, 100)
    while True:
        SUCCESS, img = cap.read()
        # findcolor(img.copy())
        
        edit(img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
def edit(img):
    # color detection in image
    def nothing(x):
        pass

    # find proper values for min and max and then you can extract the color
    cv2.namedWindow('trackbar')
    cv2.resizeWindow('trackbar', 600,600)
    cv2.createTrackbar('Hue Min', 'trackbar', 0, 179, nothing)
    cv2.createTrackbar('Hue Max', 'trackbar', 179, 179, nothing)
    cv2.createTrackbar('sat Min', 'trackbar', 0, 255, nothing)
    cv2.createTrackbar('sat Max', 'trackbar', 255, 255, nothing)
    cv2.createTrackbar('val Min', 'trackbar', 0, 255, nothing)
    cv2.createTrackbar('val Max', 'trackbar', 255, 255, nothing)

    # while True:
        
    imgHSV =  cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    h_min = cv2.getTrackbarPos('Hue Min', 'trackbar')
    h_max = cv2.getTrackbarPos('Hue Min', 'trackbar')
    sat_min = cv2.getTrackbarPos('sat Min', 'trackbar')
    sat_max = cv2.getTrackbarPos('sat Max', 'trackbar')
    val_min = cv2.getTrackbarPos('val Min', 'trackbar')
    val_max = cv2.getTrackbarPos('val Max', 'trackbar')
        
    lower = np.array([h_min, sat_min, val_min])
    higher = np.array([h_max, sat_max, val_max])
    print(h_min, h_max, sat_min, sat_max, val_min, val_max)
    mask = cv2.inRange(imgHSV, lower,higher)
        
    imgResult = cv2.bitwise_and(img, img, mask=mask)
        
        # cv2.imshow('image', imgHSV)
        # cv2.imshow('mask', mask)
        # cv2.imshow('result', imgResult)
    imgstack = stackImages(0.5,([img, imgHSV], [mask, imgResult]))
    cv2.imshow('image', imgstack)
    cv2.waitKey(1)


In [9]:
def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

In [8]:
framwidth = 640
framheight = 480

cap = cv2.VideoCapture(0)
cap.set(3, framwidth)
cap.set(4, framheight)
cap.set(10, 100)
while True:
    SUCCESS, img = cap.read()
    img_copy = img.copy()
    findcolor(img_copy)
    cv2.imshow('img', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    